In [5]:
!rm -rf /kaggle/working/distilbert-base-uncased-lora-imdb-sentiment

In [2]:
!pip install pandas==1.5.3 datasets==2.17.1 peft==0.8.2 transformers==4.37.2 torch==2.1.0 evaluate==0.4.1 accelerate==0.27.2 pyarrow==14.0.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 2.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 64.5 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 78.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 648.2 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 41.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 10.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    pipeline,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

2024-02-25 18:37:16.972461: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 18:37:16.972588: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 18:37:17.285703: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
!rm -rf data

In [10]:
!mv train.csv /content/data/train.csv
!mv test.csv /content/data/test.csv



mv: cannot stat 'train.csv': No such file or directory
mv: cannot stat 'test.csv': No such file or directory


In [4]:
imdb = load_dataset("/kaggle/input/imdb-sentiment", data_files={"train":"/kaggle/input/imdb-sentiment/train.csv", "test":"/kaggle/input/imdb-sentiment/test.csv"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
imdb['train'] = imdb['train'].class_encode_column("sentiment")
class_label_feature = imdb["train"].features["sentiment"]

imdb['test'] = imdb['test'].cast_column("sentiment", class_label_feature)

Casting to class labels:   0%|          | 0/30000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [6]:
imdb = imdb.rename_columns({"sentiment": "label", "review": "text"})

In [14]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

# define an evaluation function to pass into trainer later
def compute_metrics(p):
    logits, labels = p
    predictions = np.argmax(logits, axis=1)

    return (accuracy.compute(predictions=predictions,
                                          references=labels))

In [15]:
"""
LoRA paper:

"We find that the most critical LoRA hyperparameter is how many LoRA adapters are used in total and
that LoRA on all linear transformer block layers is required to match full finetuning performance."
"""
def find_target_modules(model):
    # Initialize a Set to Store Unique Layers
    unique_layers = set()

    # Iterate Over All Named Modules in the Model
    for name, module in model.named_modules():
        # Check if the Module Type Contains 'Linear4bit'
        if "Linear" in str(type(module)):
            # Extract the Type of the Layer
            layer_type = name.split('.')[-1]

            # Add the Layer Type to the Set of Unique Layers
            unique_layers.add(layer_type)

    # Return the Set of Unique Layers Converted to a List
    return list(unique_layers)

In [16]:
def AutoFineTune(model_name):
  id2label = {0: "negative", 1: "positive"}
  label2id = {"negative": 0, "positive": 1}

  model = AutoModelForSequenceClassification.from_pretrained(
      model_name, num_labels=2, id2label=id2label, label2id=label2id
  )
  tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

  # add pad token if none exists
  if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

  def preprocess_function(examples):
    tokenizer.truncation_side = "left"
    return tokenizer(examples["text"],
                     truncation=True,
                     max_length=512,
                     return_tensors = 'np')

  imdb_tokenized = imdb.map(preprocess_function, batched=True)

  data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  target_modules = find_target_modules(model)

  peft_config = LoraConfig(task_type="SEQ_CLS", # sequence classification
                        r=4, # intrinsic rank of trainable weight matrix
                        lora_alpha=32, # this is like a learning rate
                        lora_dropout=0.01, # probablity of dropout
                        target_modules = target_modules) # All linear layers
#                         init_lora_weights = False)

  model = get_peft_model(model, peft_config)
  model.print_trainable_parameters()

  # hyperparameters
  lr = 1e-3 # size of optimization step
  batch_size = 4 # number of examples processed per optimziation step
  num_epochs = 10 # number of times model runs through training data

  # define training arguments
  training_args = TrainingArguments(
      output_dir= model_name + "-lora-imdb-sentiment",
      learning_rate=lr,
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      num_train_epochs=num_epochs,
      weight_decay=0.01,
      evaluation_strategy="epoch",
      save_strategy="epoch",
      load_best_model_at_end=True,
      hub_private_repo = True
  )

  trainer = Trainer(
    model=model, # our peft model
    args=training_args, # hyperparameters
    train_dataset=imdb_tokenized["train"], # training data
    eval_dataset=imdb_tokenized["test"], # validation data
    tokenizer=tokenizer, # define tokenizer
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics, # evaluates model using compute_metrics() function from before
  )

  # train model
  trainer.train()

  model.save_pretrained("models")
  trainer.push_to_hub()

  return trainer

In [17]:
models = ['distilbert-base-uncased', 
          'FacebookAI/roberta-base', 
          'openai-community/gpt2']

In [ ]:
for model in models:
  print(f"-------- {model} -------")
  AutoFineTune(model)

-------- distilbert-base-uncased -------


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

trainable params: 933,130 || all params: 67,897,364 || trainable%: 1.3743243404854422


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

## Inference

In [7]:
config = PeftConfig.from_pretrained("muneeb-ai/distilbert-base-uncased-lora-imdb-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")
model = PeftModel.from_pretrained(model, "muneeb-ai/distilbert-base-uncased-lora-imdb-sentiment")

adapter_config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.safetensors:   0%|          | 0.00/3.82M [00:00<?, ?B/s]

In [25]:
tokenizer = AutoTokenizer.from_pretrained("muneeb-ai/distilbert-base-uncased-lora-imdb-sentiment", add_prefix_space=True)

# # add pad token if none exists
# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})
#     model.resize_token_embeddings(len(tokenizer))

# def preprocess_function(examples):
#     tokenizer.truncation_side = "left"
#     return tokenizer(examples["text"],
#                      truncation=True,
#                      max_length=512,
#                      return_tensors = 'np')

# imdb_tokenized = imdb.map(preprocess_function, batched=True)

In [22]:
test_df = imdb['test'].to_pandas()

In [23]:
test_df

,text,label
0,Steven Rea plays a forensic scientist thrust o...,1
1,As the first of the TV specials offered on the...,1
2,There may something poetically right in seeing...,0
3,all i can say about this film is to read the b...,0
4,I thought it was a pretty good movie and shoul...,1
...,...,...
19995,Well-done ghost story that will give you the c...,1
19996,I'm at a loss for words. This movie is beyond ...,0
19997,"First off, I had my doubts just looking at the...",0
19998,"In an early scene, Luca (David Pasquesi) and J...",1


In [32]:
id2label = {0: "negative", 1: "positive"}
label2id = {"negative": 0, "positive": 1}

In [41]:
from tqdm import tqdm

In [45]:
predictions_ls = []
model.to("cuda")
for index, row in tqdm(test_df.iterrows()):
    text = row['text']
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt", truncation="only_first").to("cuda")
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)
    
    predictions_ls.append(id2label[predictions.tolist()])

20000it [04:51, 68.71it/s]


In [46]:
test_df['predictions'] = predictions_ls

In [47]:
test_df['label'] = test_df['label'].map(id2label)

In [48]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [49]:
accuracy_score(test_df['label'], test_df['predictions'])

0.50325